In [1]:
# %%bash
# # Colab-specific setup
# !(stat -t /usr/local/lib/*/dist-packages/google/colab > /dev/null 2>&1) && exit
# pip install yacs 2>&1 >> install.log
# git init 2>&1 >> install.log
# git remote add origin https://github.com/jeongrok/semantic-segmentation-pytorch.git 2>> install.log
# git pull origin master 2>&1 >> install.log
# DOWNLOAD_ONLY=1 ./demo_test.sh 2>> install.log

In [1]:
# %pip install --default-timeout=100 future
# %pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os, csv, torch, numpy as np, scipy.io, PIL.Image, torchvision.transforms, glob, matplotlib.pyplot as plt, cv2
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/Users/yoon/miniforge3/envs/wall_ai/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/yoon/miniforge3/envs/wall_ai/lib/python3.11/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <CFED5F8E-EC3F-36FD-AAA3-2C6C7F8D3DD9> /Users/yoon/miniforge3/envs/wall_ai/lib/python3.11/site-packages/torchvision/image.so
  Expected in:     <E459C462-F863-3A5A-AC9F-FD77B14BE845> /Users/yoon/miniforge3/envs/wall_ai/lib/python3.11/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [4]:
# System libs
# import os, csv, torch, numpy as np, scipy.io, PIL.Image, torchvision.transforms, glob, matplotlib.pyplot as plt, cv2
# Our libs
from mit_semseg.models.models import ModelBuilder, SegmentationModule
from mit_semseg.utils import colorEncode

colors = scipy.io.loadmat('data/color150.mat')['colors']
names = {}
with open('data/object150_info.csv') as f:
    reader = csv.reader(f)
    next(reader)
    for row in reader:
        names[int(row[0])] = row[5].split(";")[0]

In [49]:
# Network Builders
# segmentation
net_encoder = ModelBuilder.build_encoder(
    arch='resnet50dilated',
    fc_dim=2048,
    # weights=weights_encoder
    weights='/Users/yoon/Documents/gatech/spring24/WallAI/semantic_segmentation_pytorch/ckpt/ade20k-resnet50dilated-ppm_deepsup/encoder_epoch_20.pth')
net_decoder = ModelBuilder.build_decoder(
    arch='ppm_deepsup',
    fc_dim=2048,
    num_class=150,
    # weights=weights_decoder,
    weights='/Users/yoon/Documents/gatech/spring24/WallAI/semantic_segmentation_pytorch/ckpt/ade20k-resnet50dilated-ppm_deepsup/decoder_epoch_20.pth',
    use_softmax=True)

crit = torch.nn.NLLLoss(ignore_index=-1)
segmentation_module = SegmentationModule(net_encoder, net_decoder, crit)#.to(DEVICE)
segmentation_module.eval()
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
segmentation_module.to(DEVICE)

Loading weights for net_encoder
Loading weights for net_decoder


TypeError: SegmentationModule.__init__() takes 3 positional arguments but 4 were given

In [51]:
#pspnet
weights_encoder = '/Users/yoon/Documents/gatech/spring24/WallSegmentation/model_weights/Transfer learning - entire decoder/transfer_encoder.pth'
weights_decoder = '/Users/yoon/Documents/gatech/spring24/WallSegmentation/model_weights/Transfer learning - entire decoder/transfer_decoder.pth'

pspnet_encoder = ModelBuilder.build_encoder(
    arch='resnet50dilated',
    fc_dim=2048,
    # weights=weights_encoder
    weights='/Users/yoon/Documents/gatech/spring24/WallAI/semantic_segmentation_pytorch/ckpt/ade20k-resnet50dilated-ppm_deepsup/encoder_epoch_20.pth')
pspnet_decoder = ModelBuilder.build_decoder(
    arch='ppm_deepsup',
    fc_dim=2048,
    num_class=150,
    # weights=weights_decoder,
    weights='/Users/yoon/Documents/gatech/spring24/WallAI/semantic_segmentation_pytorch/ckpt/ade20k-resnet50dilated-ppm_deepsup/decoder_epoch_20.pth',
    use_softmax=True)

crit = torch.nn.NLLLoss(ignore_index=-1)
segmentation_module = SegmentationModule(net_encoder, net_decoder, crit)#.to(DEVICE)
segmentation_module.eval()
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
segmentation_module.to(DEVICE)

Loading weights for net_encoder
Loading weights for net_decoder


TypeError: SegmentationModule.__init__() takes 3 positional arguments but 4 were given

In [5]:
# from google.colab import drive
# drive.mount('/content/gdrive/', force_remount=True)
# %cd /content/gdrive/My Drive/CV Project

Mounted at /content/gdrive/
/content/gdrive/My Drive/CV Project


In [41]:
#pspnet
os.chdir('/Users/yoon/Documents/gatech/spring24/WallAI/WallSegmentation')
from models.models import SegmentationModule, build_encoder, build_decoder
from src.eval import segment_image

# Model weights (encoder and decoder)
weights_encoder = '/Users/yoon/Documents/gatech/spring24/WallSegmentation/model_weights/Transfer learning - entire decoder/transfer_encoder.pth'
weights_decoder = '/Users/yoon/Documents/gatech/spring24/WallSegmentation/model_weights/Transfer learning - entire decoder/transfer_decoder.pth'


net_encoder = build_encoder(weights_encoder)
net_decoder = build_decoder(weights_decoder)

psp_segmentation_module = SegmentationModule(net_encoder, net_decoder)
psp_segmentation_module = psp_segmentation_module.to(DEVICE).eval()
os.chdir('/Users/yoon/Documents/gatech/spring24/WallAI/semantic_segmentation_pytorch')

Building encoder: resnet50-dilated
Loading weights for net_encoder
Loading weights for net_decoder


In [42]:
# Load and normalize one image as a singleton tensor batch
pil_to_tensor = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(
        mean=[0.485, 0.456, 0.406], # These are RGB mean+std values
        std=[0.229, 0.224, 0.225])  # across a large photo dataset.
])

In [43]:
def readImage(img_name, psp):
    pil_image = PIL.Image.open(img_name).convert('RGB')
    img_original = np.array(pil_image)
    img_data = pil_to_tensor(pil_image)

    
    singleton_batch = {'img_data': img_data[None].to(DEVICE)}
    
    output_size = img_data.shape[1:]
    with torch.no_grad():
        if psp:
             img = segment_image(segmentation_module, img_original)
            #  scores = psp_segmentation_module(singleton_batch, segSize=output_size)
        else:
            scores = segmentation_module(singleton_batch, segSize=output_size)

    # Get the predicted scores for each pixel
    _, model_pred = torch.max(scores, dim=1)
    model_pred = model_pred.cpu()[0].numpy()
    model_pred[model_pred != 0] = -1

    model_pred = colorEncode(model_pred, colors).astype(np.uint8)
    return img_original, model_pred

def resizeAndPad(img):
    height, width = img.shape[:2]
    scaled_height, scaled_width = height+2, width+2
    interpolate = cv2.INTER_CUBIC
    aspect = width/height
    if aspect > 1:
        new_width = scaled_width
        new_height = np.round(new_width/aspect).astype(int)
        pad_vertical = (scaled_height-new_height)/2
        pad_top, pad_bottom = np.floor(pad_vertical).astype(int), np.ceil(pad_vertical).astype(int)
        pad_left, pad_right = 0, 0
    elif aspect < 1:
        new_height = scaled_height
        new_width = np.round(new_height*aspect).astype(int)
        pad_horizontal = (scaled_width-new_width)/2
        pad_left, pad_right = np.floor(pad_horizontal).astype(int), np.ceil(pad_horizontal).astype(int)
        pad_top, pad_bottom = 0, 0
    else:
        new_height, new_width = scaled_height, scaled_width
        pad_left, pad_right, pad_top, pad_bottom = 0, 0, 0, 0
    pad_color = [255]*3
    scaled_img = cv2.resize(img, (new_width, new_height), interpolation=interpolate)
    scaled_img = cv2.copyMakeBorder(scaled_img, pad_top, pad_bottom, pad_left, pad_right, borderType=cv2.BORDER_CONSTANT, value=pad_color)
    return scaled_img

def getOutlineImg(img):
    # gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return cv2.Canny(img,60,180)

def getColoredImage(img, new_color):
    hsv_img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    h, s, v = cv2.split(hsv_img)
    hsv_color = cv2.cvtColor(np.uint8([[new_color]]), cv2.COLOR_RGB2HSV)
    h.fill(hsv_color[0][0][0])
    s.fill(hsv_color[0][0][1])
    new_hsv_img = cv2.merge([h, s, v])
    new_rgb_img = cv2.cvtColor(new_hsv_img, cv2.COLOR_HSV2RGB)
    return new_rgb_img

def selectWall(outline_img, position):
    wall = outline_img.copy()
    scaled_mask = resizeAndPad(outline_img)
    for p in position:
        cv2.floodFill(wall, scaled_mask, p, 255)
    cv2.subtract(wall, outline_img, wall)
    return wall

def getSamples(mask):
    gray = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    population = np.argwhere(gray == 120)
    s = np.log(len(population)).astype(int)
    i = np.random.choice(len(population), s, replace=False)
    final_inds = population[i]
    final_inds = [(y,x) for x,y in final_inds]
    return final_inds

def mergeMasks(mask1, mask2):
    img = cv2.bitwise_or(mask1, mask2)
    return img

def mergeImages(img, colored_img, wall):
    colored_img = cv2.bitwise_and(colored_img, colored_img, mask=wall)
    marked_img = cv2.bitwise_and(img, img, mask=cv2.bitwise_not(wall))
    final_img = cv2.bitwise_xor(colored_img, marked_img)
    return final_img

def saveImage(img_name, img):
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    img_name= img_name[8:]
    cv2.imwrite("./resnetcv/" + img_name, img)

def changeColor(img_name, new_color, psp=False):
    img_original, model_pred = readImage(img_name, psp)
    colored_img = getColoredImage(img_original, new_color)
    outline_img = getOutlineImg(img_original)
    pred_canny = getOutlineImg(model_pred)
    merge_mask = mergeMasks(pred_canny, outline_img)
    inds = getSamples(model_pred)
    selected_wall = selectWall(merge_mask, inds)
    final_img = mergeImages(img_original, colored_img, selected_wall)
    saveImage(img_name, final_img)
    return final_img, selected_wall

def stats(pred_mask, true_mask):
    total_pixels = pred_mask.shape[0] * pred_mask.shape[1]
    print(f'total of pred: {total_pixels}')
    print(f'total of true: {true_mask.shape[0] * true_mask.shape[1]}')
    pred_mask = np.where(pred_mask == 255, 1, 0)
    print('weifoew: ', pred_mask)
    true_mask = np.where(true_mask == 255, 1, 0)
    accuracy = np.sum(pred_mask == true_mask) / total_pixels
    precision = np.sum(pred_mask & true_mask) / np.sum(pred_mask)
    recall = np.sum(pred_mask & true_mask) / np.sum(true_mask)
    return accuracy, precision, recall

def IOU(pred_mask, true_mask):
    """
        Function for calculating IOU of an image
    """

    pred_mask = np.where(pred_mask == 255, 1, 0)
    true_mask = np.where(true_mask == 255, 1, 0)

    intersection = np.sum((pred_mask == 1) & (true_mask == 1))
    union = np.sum((pred_mask == 1) | (true_mask == 1))
    
    # ((pred_mask == 0) + (true_mask == 0)).sum() + 1e-15  # protection from division with 0

    return intersection / (union + 1e-15)


def combine_masks(masks):
    combined_mask = masks[0]
    for mask in masks[1:]:
        combined_mask = cv2.bitwise_or(combined_mask, mask)
    return combined_mask

from matplotlib import pyplot

In [15]:
# segmentation
new_color = [111, 209, 201]
total_accuracy = 0
total_precision = 0
total_recall = 0
total_iou = 0
for i in range(1,101):
    # m = 'bedroom/' + str(i) + 'wall*'
    m = '/Users/yoon/Documents/gatech/spring24/WallAI/Data/bedroom/' + str(i) + 'wall*'
    masks = glob.glob(m)
    # print(masks)
    # j = 'bedroom/' + str(i) + '_img*'
    j = '/Users/yoon/Documents/gatech/spring24/WallAI/Data/bedroom/' + str(i) + '_img*'
    img = glob.glob(j)
    masks = [cv2.imread(mask) for mask in masks]
    masks = [cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY) for mask in masks]
    final_mask = masks[0]
    if len(masks) > 1:
        for mask in masks[1:]:
            final_mask = cv2.bitwise_or(final_mask, mask)

    final_img, wall = changeColor(img[0],new_color, psp=False)
    
    acc, prec, rec = stats(wall, final_mask)
    total_accuracy += acc
    total_precision += prec
    total_recall += rec

    iou = IOU(wall, final_mask)
    print(f'iteration {i} iou: {iou}')
    # break
    total_iou += iou

    if i % 20 == 0:
        print('iteration: ', i)

total_accuracy /= 100
total_precision /= 100
total_recall /= 100
total_iou /= 100
print('Accuracy:', total_accuracy)
print('Precision: ', total_precision)
print('Recall: ', total_recall)
print('F1: ', 2 * (total_precision * total_recall) / (total_precision + total_recall))
print('IOU: ', total_iou)

iteration 1 iou: 0.2281711231869306
iteration 2 iou: 0.9403951880681509
iteration 3 iou: 0.7884208434171607
iteration 4 iou: 0.954853055959066
iteration 5 iou: 0.7159037340667643
iteration 6 iou: 0.7619780404257616
iteration 7 iou: 0.8281981563399526
iteration 8 iou: 0.2637907684876088
iteration 9 iou: 0.5934304354181629
iteration 10 iou: 0.8260631763935837
iteration 11 iou: 0.4603055749760326
iteration 12 iou: 0.47312454883669275
iteration 13 iou: 0.7353166154280898
iteration 14 iou: 0.6227172578430156
iteration 15 iou: 0.8909820862583053
iteration 16 iou: 0.7301503094606543
iteration 17 iou: 0.4291044776119403
iteration 18 iou: 0.5814810011671638
iteration 19 iou: 0.5247390597890447
iteration 20 iou: 0.930673014371045
iteration:  20
iteration 21 iou: 0.8296782469643453
iteration 22 iou: 0.9271442945100263
iteration 23 iou: 0.6448712467523925
iteration 24 iou: 0.6095305981440828
iteration 25 iou: 0.7453907346747777
iteration 26 iou: 0.7161987207976658
iteration 27 iou: 0.8801581490701

In [48]:
from PIL import Image
IMAGENET_MEAN = np.array([0.485, 0.456, 0.406], dtype=np.float32)
IMAGENET_STD = np.array([0.229, 0.224, 0.225], dtype=np.float32)



def visualize_wall(img, pred, class_to_display=0):
    """
        Function for visualizing wall prediction 
        (original image, segmentation mask and original image with the segmented wall)
    """
    img_green = img.copy()

    wall_pixels = pred == class_to_display
    
    # Retain original pixels where it's not a wall
    img_green[~wall_pixels] = img[~wall_pixels]
    
    # Create a black background
    black_background = np.zeros_like(img)
    
    # Color the wall pixels with green
    black_background[wall_pixels] = [255, 0, 0]
    black_background[~wall_pixels] = [0,0,0]
    # black_background[wall_pixels] = [111, 209, 201]

    return np.array(PIL.Image.fromarray(black_background))
    
def segment_image(segmentation_module, img, disp_image=True):
    """
        Function for segmenting wall in the input image. The input can be path to image, or a loaded image
    """
    pil_to_tensor = torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
    ])

    if isinstance(img, str):
        img = Image.open(img)
    
    img_original = np.array(img)
    img_data = pil_to_tensor(img)
    singleton_batch = {'img_data': img_data[None].to(DEVICE)}
    seg_size = img_original.shape[:2]

    with torch.no_grad():
        scores = segmentation_module(singleton_batch, seg_size=seg_size)

    _, pred = torch.max(scores, dim=1)
    pred = pred.cpu()[0].numpy()

    if disp_image:
        return visualize_wall(img_original, pred)
    else:
        print("WEOIFJEWOIFJ")
        
    return pred

In [52]:
#pspnet

def stats1(pred_mask, true_mask):
    total_pixels = pred_mask.shape[0] * pred_mask.shape[1]
    # print(f'total of pred: {total_pixels}')
    # print(f'total of true: {true_mask.shape[0] * true_mask.shape[1]}')
    pred_mask = np.where(pred_mask == 255, 1, 0)
    # print('weifoew: ', pred_mask)
    true_mask = np.where(true_mask == 255, 1, 0)
    accuracy = np.sum(pred_mask == true_mask) / total_pixels
    precision = np.sum(pred_mask & true_mask) / np.sum(pred_mask)
    recall = np.sum(pred_mask & true_mask) / np.sum(true_mask)
    return accuracy, precision, recall

new_color = [111, 209, 201]
total_accuracy = 0
total_precision = 0
total_recall = 0
total_iou = 0
for i in range(1,101):
# for i in range(1,11):
    # m = 'bedroom/' + str(i) + 'wall*'
    m = '/Users/yoon/Documents/gatech/spring24/WallAI/Data/bedroom/' + str(i) + 'wall*'
    masks = glob.glob(m)
    # print(masks)
    # j = 'bedroom/' + str(i) + '_img*'
    j = '/Users/yoon/Documents/gatech/spring24/WallAI/Data/bedroom/' + str(i) + '_img*'
    img = glob.glob(j)
    masks = [cv2.imread(mask) for mask in masks]
    masks = [cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY) for mask in masks]
    final_mask = masks[0]
    if len(masks) > 1:
        for mask in masks[1:]:
            final_mask = cv2.bitwise_or(final_mask, mask)

    
    # final_img, wall = changeColor(img[0], new_color)

    wall = segment_image(psp_segmentation_module, img[0])
    wall = wall[:,:,0]
    # wall = cv2.imread(wall)
    # wall = cv2.cvtColor(wall, cv2.COLOR_BGR2GRAY)

    # pyplot.imshow(final_mask)
    # break

    
    acc, prec, rec = stats1(wall, final_mask)
    total_accuracy += acc
    total_precision += prec
    total_recall += rec

    iou = IOU(wall, final_mask)
    total_iou += iou
    # break
    if i % 10 == 0:
        print('iteration: ', i)

total_accuracy /= 100
total_precision /= 100
total_recall /= 100
total_iou /= 100
print('Accuracy:', total_accuracy)
print('Precision: ', total_precision)
print('Recall: ', total_recall)
print('F1: ', 2 * (total_precision * total_recall) / (total_precision + total_recall))
print('IOU: ', total_iou)

ffweafwaefwef
torch.Size([1, 2048, 207, 275])
ffweafwaefwef
torch.Size([1, 2048, 106, 150])
ffweafwaefwef
torch.Size([1, 2048, 96, 128])
ffweafwaefwef
torch.Size([1, 2048, 150, 200])
ffweafwaefwef
torch.Size([1, 2048, 192, 256])
ffweafwaefwef
torch.Size([1, 2048, 108, 144])
ffweafwaefwef
torch.Size([1, 2048, 150, 200])
ffweafwaefwef
torch.Size([1, 2048, 78, 117])
ffweafwaefwef
torch.Size([1, 2048, 192, 256])
ffweafwaefwef
torch.Size([1, 2048, 84, 128])
iteration:  10
ffweafwaefwef
torch.Size([1, 2048, 207, 275])
ffweafwaefwef
torch.Size([1, 2048, 62, 94])
ffweafwaefwef
torch.Size([1, 2048, 75, 113])
ffweafwaefwef
torch.Size([1, 2048, 192, 256])
ffweafwaefwef
torch.Size([1, 2048, 120, 160])
ffweafwaefwef
torch.Size([1, 2048, 67, 100])
ffweafwaefwef
torch.Size([1, 2048, 32, 32])
ffweafwaefwef
torch.Size([1, 2048, 38, 57])
ffweafwaefwef
torch.Size([1, 2048, 150, 100])
ffweafwaefwef
torch.Size([1, 2048, 109, 164])
iteration:  20
ffweafwaefwef
torch.Size([1, 2048, 60, 80])
ffweafwaefwef
tor

In [37]:
#original

new_color = [111, 209, 201]
total_accuracy = 0
total_precision = 0
total_recall = 0
for i in range(1,101):
    m = 'bedroom/' + str(i) + 'wall*'
    masks = glob.glob(m)
    j = 'bedroom/' + str(i) + '_img*'
    img = glob.glob(j)
    masks = [cv2.imread(mask) for mask in masks]
    masks = [cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY) for mask in masks]
    final_mask = masks[0]
    if len(masks) > 1:
        for mask in masks[1:]:
            final_mask = cv2.bitwise_or(final_mask, mask)

    final_img, wall = changeColor(img[0],new_color)
    acc, prec, rec = stats(wall, final_mask)
    total_accuracy += acc
    total_precision += prec
    total_recall += rec
    if i % 20 == 0:
        print('iteration: ', i)

total_accuracy /= 100
total_precision /= 100
total_recall /= 100
print('Accuracy:', total_accuracy)
print('Precision: ', total_precision)
print('Recall: ', total_recall)
print('F1: ', 2 * (total_precision * total_recall) / (total_precision + total_recall))

iteration:  20
iteration:  40
iteration:  60
iteration:  80
iteration:  100
Accuracy: 0.8930769958447106
Precision:  0.8607124761024225
Recall:  0.8277385887280082
F1:  0.8439035576564474
